# 💬 Fine-tuning d’un modèle LLM avec LoRA sur un dataset de citations


Ce notebook a pour but de te montrer comment utiliser LoRA (Low-Rank Adaptation) pour fine-tuner un modèle de langage causal (`CausalLM`) en utilisant un petit dataset de citations inspirantes.

Nous allons utiliser le modèle `bigscience/bloomz-560m` et le dataset `Abirate/english_quotes` (10% d’échantillons).


## 1. Installation des bibliothèques nécessaires

In [1]:

%pip install peft==0.4.0
%pip install datasets transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

## 2. Chargement du dataset

In [4]:
from datasets import load_dataset

# On charge 10% de l'échantillon d'entraînement
# The split="train[:10%]" was causing a ValueError.
# Load the full training split first, then select the first 10%.
# Trying to load the dataset without specifying a split initially
dataset_dict = load_dataset("Abirate/english_quotes")
dataset = dataset_dict['train'] # Access the train split from the dictionary
dataset = dataset.select(range(int(len(dataset) * 0.1))) # Select the first 10%
dataset = dataset.shuffle(seed=42)
dataset = dataset.select(range(20))  # Sous-échantillon pour le test
dataset

ValueError: Invalid pattern: '**' can only be an entire path component

## 3. Chargement du tokenizer et du modèle de base

In [5]:

from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "bigscience/bloomz-560m"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

## 4. Prétraitement : tokenisation du texte

In [7]:

def tokenize_function(sample):
    return tokenizer(sample["quote"], truncation=True, padding="max_length", max_length=64)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
tokenized_dataset


NameError: name 'dataset' is not defined

## 5. Configuration LoRA et application

In [8]:

from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

peft_model = get_peft_model(model, lora_config)
peft_model.print_trainable_parameters()


trainable params: 393,216 || all params: 559,607,808 || trainable%: 0.07026635339584111


## 6. Entraînement du modèle avec Trainer

In [10]:

from transformers import TrainingArguments, Trainer, DataCollatorForLanguageModeling
import os

output_dir = "./lora_quotes_outputs"
training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=2,
    num_train_epochs=1,
    learning_rate=3e-4,
    report_to="none",
    save_strategy="no"
)

trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

trainer.train()


NameError: name 'tokenized_dataset' is not defined

## 7. Sauvegarde du modèle LoRA entraîné

In [11]:

from datetime import datetime
from peft import PeftModel

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
save_path = os.path.join(output_dir, f"peft_model_{timestamp}")
peft_model.save_pretrained(save_path)
print(f"Modèle LoRA sauvegardé dans : {save_path}")


Modèle LoRA sauvegardé dans : ./lora_quotes_outputs/peft_model_20250722_130532


## 8. Inférence avec le modèle fine-tuné

In [12]:

from peft import PeftModel
from transformers import pipeline

# Rechargement du modèle
model_infer = PeftModel.from_pretrained(model, save_path, is_trainable=False)

# Pipeline pour génération de texte
text_generator = pipeline("text-generation", model=model_infer, tokenizer=tokenizer)

prompt = "Life is like"
outputs = text_generator(prompt, max_new_tokens=30, do_sample=True)
print(outputs[0]["generated_text"])


Device set to use cpu


Life is like the game of Chess, you have to beat it
